In [ ]:
import pandas as pd
import sqlite3

# Cargar datasets desde la carpeta data
df_supermercado = pd.read_csv("../data/Supermercado.csv")
df_ventas = pd.read_csv("../data/Ventas.csv")
df_inventario = pd.read_csv("../data/Inventario.csv")

In [ ]:
# Verificar registros
print(df_supermercado.shape)
display(df_supermercado.head(10))
print(df_ventas.shape)
display(df_ventas.head(10))
print(df_inventario.shape)
display(df_inventario.head(10))

## Transformación de los Datos


In [ ]:
# Eliminar duplicados

# Supermercado
df_supermercado.drop_duplicates(inplace=True)

# Ventas
df_ventas.drop_duplicates(inplace=True)

# Categoria
df_inventario.drop_duplicates(inplace=True)

## Unificación de string/textos a minúsculas para evitar inconsistencias
# Cambiar los datos string a minúsculas para evitar problemas de inconsistencia
for df in [df_supermercado, df_ventas, df_inventario]:
    for col in df.select_dtypes(include=["object"]).columns:
        df[col] = df[col].str.lower().str.strip()

# Convertir fecha a datetime en los tres DataFrames
for df in (df_ventas, df_inventario):
    if "Fecha" in df:
        df["Fecha"] = pd.to_datetime(df["Fecha"], errors="coerce")
    for c in ("monto_total", "cantidad_vendida"):
        if c in df:
            df[c] = pd.to_numeric(df[c], errors="coerce")

# Convertir valores float a Int en DataFrane Ventas e Inventario
# Ventas
df_ventas["Cantidad_Vendida"] = df_ventas["Cantidad_Vendida"].astype("Int64")

# Inventario
df_inventario["Lead_Time_Dias"] = df_inventario["Lead_Time_Dias"].astype("Int64")
df_inventario["Stock_Minimo"] = df_inventario["Stock_Minimo"].astype("Int64")
df_inventario["Stock_Objetivo"] = df_inventario["Stock_Objetivo"].astype("Int64")
df_inventario["Stock_Disponible"] = df_inventario["Stock_Disponible"].astype("Int64")
df_inventario["Pedido_Pendiente"] = df_inventario["Pedido_Pendiente"].astype("Int64")

In [ ]:
# Borrar valores nulos si existen
df_supermercado.dropna(inplace=True)
df_ventas.dropna(inplace=True)
df_inventario.dropna(inplace=True)

print("Limpieza de datos completa.")

In [ ]:
# Verificacón de Datos Generales
print("VALIDACIÓN: SUPERMERCADO")
print("Tipos de datos:\n", df_supermercado.dtypes)
print("Nulos:\n", df_supermercado.isnull().sum())
print("Duplicados:", df_supermercado.duplicated().sum())
print("IDs únicos:", df_supermercado.index.is_unique)
print("------------------------------- \n")
print("VALIDACIÓN: VENTAS")
print("Tipos de datos:\n", df_ventas.dtypes)
print("Nulos:\n", df_ventas.isnull().sum())
print("Duplicados:", df_ventas.duplicated().sum())
print("IDs únicos:", df_ventas.index.is_unique)
print("------------------------------- \n")
print("VALIDACIÓN: CATEGORÍA")
print("Tipos de datos:\n", df_inventario.dtypes)
print("Nulos:\n", df_inventario.isnull().sum())
print("Duplicados:", df_inventario.duplicated().sum())
print("IDs únicos:", df_inventario.index.is_unique)
print("------------------------------- \n")

# Verificamos las relaciones entre las tablas/entidades
# PK de Entidades Supermercado - Ventas - Inventario

# Supermercado
print("PK Supermercado: Id_Tienda")
print(df_supermercado["Id_Tienda"].is_unique)

# Ventas
print("PK Ventas: Id_Venta")
print(df_ventas["Id_Venta"].is_unique)

# Inventario
print("PK Inventario: Id_Inventario")
print(df_inventario["Id_Inventario"].is_unique)

print("------------------------------- \n")

# FK de Entidades Ventas - Inventario - Supermercado

# Ventas -> Supermercado
print("FK Ventas -> Supermercado: Id_Tienda")
df_ventas["Id_Tienda"].isin(df_supermercado["Id_Tienda"]).value_counts()

# Inventario -> Supermercado
print("FK Inventario -> Supermercado: Id_Tienda")
df_inventario["Id_Tienda"].isin(df_supermercado["Id_Tienda"]).value_counts()
print("-------------------------------")

In [ ]:
# La cantidad de registros en cada entidad
print(
    f"TOTAL REGISTROS :\n"
    f"Supermercado: {len(df_supermercado)}, "
    f"Ventas: {len(df_ventas)}, "
    f"Inventario: {len(df_inventario)}"
)

In [ ]:
# Selección de filas relevantes y exportación a nuevos CSV limpios y posterior creación de base de datos SQLite
# Seleccionamos las columnas relevantes para cada entidad (Clustering)
supermercado = df_supermercado[["Id_Tienda", "Nombre_Supermercado", "Ciudad"]]
ventas = df_ventas[
    [
        "Id_Venta",
        "Fecha",
        "Nombre_Supermercado",
        "Ciudad",
        "Categoria",
        "Cantidad_Vendida",
        "Monto_Total",
        "Id_Tienda",
    ]
]
inventario = df_inventario[
    [
        "Id_Inventario",
        "Fecha",
        "Id_Tienda",
        "Categoria",
        "Promocion",
        "Precio",
        "Descuento_pct",
        "Lead_Time_Dias",
        "Stock_Minimo",
        "Stock_Objetivo",
        "Stock_Disponible",
        "Pedido_Pendiente",
        "Rotura_Stock",
    ]
]

In [ ]:
# Conectamos a la base de datos SQLite y en caso de no existir se creará la nueva base de datos
conn = sqlite3.connect("../database/dodo_supermercado.db")

# Cargamos los DataFrames a la base de datos
# Usamos if_exists='replace' para reemplazar las tablas si ya existen y así no tener problemas al ejecutar varias veces el script
supermercado.to_sql("Supermercado", conn, if_exists="replace", index=False)
ventas.to_sql("Ventas", conn, if_exists="replace", index=False)
inventario.to_sql("Inventario", conn, if_exists="replace", index=False)

print("ETL finalizado, BD creada en ../database/dodo_supermercado.db")